In [1]:
import json

In [2]:
with open('replay_parser/replay_data.json', 'r') as file:
    replay_data = json.load(file)

In [3]:
replay_data['properties']['Goals']

[{'frame': 699, 'PlayerName': 'KittyAction', 'PlayerTeam': 0},
 {'frame': 1179, 'PlayerName': 'sharkyblan', 'PlayerTeam': 1},
 {'frame': 2167, 'PlayerName': 'TT zerrpex', 'PlayerTeam': 0},
 {'frame': 2803, 'PlayerName': 'TT zerrpex', 'PlayerTeam': 0},
 {'frame': 4593, 'PlayerName': 'UnclearSquash54', 'PlayerTeam': 0},
 {'frame': 5242, 'PlayerName': 'omarh8m', 'PlayerTeam': 1},
 {'frame': 6287, 'PlayerName': 'omarh8m', 'PlayerTeam': 1},
 {'frame': 8188, 'PlayerName': 'ily yaz', 'PlayerTeam': 1},
 {'frame': 9160, 'PlayerName': 'TT zerrpex', 'PlayerTeam': 0},
 {'frame': 10596, 'PlayerName': 'ily yaz', 'PlayerTeam': 1},
 {'frame': 11012, 'PlayerName': 'UnclearSquash54', 'PlayerTeam': 0}]

In [4]:
replay_data['objects'][275]

'TAGame.Vehicle_TA:bDriving'

In [5]:
replay_data['objects'].index('TAGame.Car_TA')

292

In [6]:
replay_data['network_frames']['frames'][2931]

{'time': 124.90641,
 'delta': 0.0333338,
 'new_actors': [],
 'deleted_actors': [],
 'updated_actors': [{'actor_id': 125,
   'stream_id': 51,
   'object_id': 271,
   'attribute': {'Byte': 241}}]}

In [7]:
replay_data['network_frames']['frames'][2976]

{'time': 126.40606,
 'delta': 0.033334,
 'new_actors': [],
 'deleted_actors': [],
 'updated_actors': [{'actor_id': 249,
   'stream_id': 50,
   'object_id': 270,
   'attribute': {'Byte': 128}},
  {'actor_id': 125,
   'stream_id': 51,
   'object_id': 271,
   'attribute': {'Byte': 249}}]}

In [8]:
FIVE_SECONDS = int(replay_data['properties']['RecordFPS'] * 5) + 5 # Some extra frames for interpolation
FIVE_SECONDS

155

In [9]:
def name_to_id(name: str) -> int:
    if name == 'None':
        raise ValueError('Name is None')
    return replay_data['names'].index(name)

In [10]:
keyframes = [keyframe_data['frame'] for keyframe_data in replay_data['keyframes']]

In [11]:
def to_recent_keyframe(frame: int) -> int:
    return min(keyframes, key=lambda keyframe: frame - keyframe if keyframe <= frame else float('inf'))

In [12]:
goals = [highlight for highlight in replay_data['properties']['HighLights'] if highlight['GoalActorName'] != 'None']
goals

[{'frame': 699,
  'CarName': 'Car_TA_261',
  'BallName': 'None',
  'GoalActorName': 'GoalVolume_TA_4'},
 {'frame': 1179,
  'CarName': 'Car_TA_275',
  'BallName': 'None',
  'GoalActorName': 'GoalVolume_TA_0'},
 {'frame': 2167,
  'CarName': 'Car_TA_288',
  'BallName': 'Ball_TA_46',
  'GoalActorName': 'GoalVolume_TA_4'},
 {'frame': 2803,
  'CarName': 'Car_TA_301',
  'BallName': 'Ball_TA_48',
  'GoalActorName': 'GoalVolume_TA_4'},
 {'frame': 4593,
  'CarName': 'Car_TA_313',
  'BallName': 'Ball_TA_50',
  'GoalActorName': 'GoalVolume_TA_4'},
 {'frame': 5242,
  'CarName': 'Car_TA_331',
  'BallName': 'Ball_TA_52',
  'GoalActorName': 'GoalVolume_TA_0'},
 {'frame': 6287,
  'CarName': 'Car_TA_350',
  'BallName': 'Ball_TA_54',
  'GoalActorName': 'GoalVolume_TA_0'},
 {'frame': 8188,
  'CarName': 'Car_TA_363',
  'BallName': 'Ball_TA_56',
  'GoalActorName': 'GoalVolume_TA_0'},
 {'frame': 9160,
  'CarName': 'Car_TA_376',
  'BallName': 'Ball_TA_58',
  'GoalActorName': 'GoalVolume_TA_4'},
 {'frame': 105

In [13]:
name_to_id(goals[0]['CarName'])

27

In [14]:
goal_data = goals[0]

In [15]:
goal_data['frame'] - FIVE_SECONDS

544

In [16]:
relevant_keyframes = [keyframe for keyframe in keyframes if to_recent_keyframe(goal_data['frame'] - FIVE_SECONDS) <= keyframe <= goal_data['frame']]
relevant_keyframes

[300, 600]

In [17]:
def get_relevant_keyframes(goal_data: dict) -> list:
    return [keyframe for keyframe in keyframes if to_recent_keyframe(goal_data['frame'] - FIVE_SECONDS) <= keyframe <= goal_data['frame']]


In [18]:
ball_names = [x for x in replay_data['names'] if 'Ball' in x]
ball_idxs = [replay_data['names'].index(ball_name) for ball_name in ball_names]
ball_idxs

[2, 139, 181, 204, 247, 350, 412, 461, 518, 572, 597, 666]

In [19]:
def handle_ball_error(keyframe: int, goal_data: dict) -> int:
    possible_ball_idx = []
    for actor in replay_data['network_frames']['frames'][keyframe]['new_actors']:
        if actor['name_id'] in ball_idxs:
            possible_ball_idx.append(actor['name_id'])
    if len(possible_ball_idx) == 1:
        return possible_ball_idx[0]
    else:
        raise ValueError('Multiple balls found')

In [20]:
# THIS REQUIRES THE LIST `keyframes` TO EXIST

def get_actors(keyframe: int, goal_data: dict) -> dict:
    """Get the actors for a given keyframe for the ball and scorer name

    Args:
        keyframe (int): A keyframe
        goal_data (dict): The goal data
    Returns:
        dict: ball and car actor idds
    """
    if keyframe not in keyframes:
        raise ValueError('Keyframe not in keyframes')
    actors = {}
    # Check actor_id when keyframe regenerates actor_ids
    new_actors = replay_data['network_frames']['frames'][keyframe]['new_actors']
    for actor in new_actors:
        try:
            ball_id = name_to_id(goal_data['BallName'])
        except ValueError:
            ball_id = handle_ball_error(keyframe, goal_data)
        car_id = name_to_id(goal_data['CarName'])
        if actor['name_id'] == ball_id:
            actors['ball'] = actor['actor_id']
        elif actor['name_id'] == car_id:
            actors['car'] = actor['actor_id']
            
    # handle ball error
    if 'ball' not in actors:
        ball_id = handle_ball_error(keyframe, goal_data)
        for actor in new_actors:
            if actor['name_id'] == ball_id:
                actors['ball'] = actor['actor_id']
    
    # final error check
    if 'ball' not in actors:
        raise ValueError('Ball not found')
    elif 'car' not in actors:
        raise ValueError('Car not found')
    return actors

In [21]:
actors = {}
for keyframe in relevant_keyframes:
    actors[keyframe] = get_actors(keyframe, goal_data)
actors


{300: {'ball': 2, 'car': 27}, 600: {'ball': 2, 'car': 27}}

In [22]:
def get_carball_data(actor_data: list, actors: dict) -> dict:
    output = {}
    for actor in actor_data:
        if actor['actor_id'] == actors['ball']:
            try:
                output['ball'] = actor['attribute']['RigidBody']
            except KeyError: # Sometimes the attribute being updated isn't the rigid body
                continue
        elif actor['actor_id'] == actors['car']:
            try:
                output['car'] = actor['attribute']['RigidBody']
            except KeyError:
                continue
            
    return output

In [23]:
def interpolate_missing_data(goal_snippet_data: dict) -> dict:
    """Interpolate missing car and ball data between known points.
    Handles:
    - Kickoff positions for ball and car at start
    - Normal interpolation between known points
    - Demolished car state for large gaps at end
    """
    interpolated_data = goal_snippet_data.copy()
    max_frame = FIVE_SECONDS
    
    # Default values
    KICKOFF_BALL_DATA = {
        'sleeping': False,
        'location': {'x': 0, 'y': 0, 'z': 110},
        'rotation': {'x': 0, 'y': 0, 'z': 0, 'w': 1},
        'linear_velocity': {'x': 0, 'y': 0, 'z': 0},
        'angular_velocity': {'x': 0, 'y': 0, 'z': 0}
    }
    
    KICKOFF_CAR_DATA = {
        'sleeping': False,
        'location': {
            'x': 2160.300000000003,
            'y': 2672.3000000000175,
            'z': 17.00999999999999
        },
        'rotation': {
            'x': -0.00538131,
            'y': -0.0018153489999999939,
            'z': 0.9238633999999948,
            'w': -0.38268401999999924
        },
        'linear_velocity': {
            'x': 2624.95,
            'y': 2624.95,
            'z': 3.7300000000000004
        },
        'angular_velocity': {
            'x': 0.3400000000000001,
            'y': 0.050000000000000044,
            'z': 0.0
        }
    }
    
    DEMOLISHED_CAR_DATA = KICKOFF_CAR_DATA.copy()
    DEMOLISHED_CAR_DATA['sleeping'] = True
    
    # Normal interpolation
    for entity in ['car', 'ball']:
        known_frames = [
            frame for frame, data in goal_snippet_data.items() 
            if entity in data
        ]
        
        if not known_frames:
            continue
            
        # Interpolate between known frames
        for i in range(len(known_frames) - 1):
            start_frame = known_frames[i]
            end_frame = known_frames[i + 1]
            frame_gap = end_frame - start_frame
            
            if frame_gap <= 1:
                continue
            
            if entity == 'car' and frame_gap > 10:
                # Use the next known position (post-respawn) with sleeping=True
                demolished_car_data = goal_snippet_data[end_frame]['car'].copy()
                demolished_car_data['sleeping'] = True
                
                # Fill in all frames in the gap
                for frame in range(start_frame + 1, end_frame):
                    if frame not in interpolated_data:
                        interpolated_data[frame] = {}
                    interpolated_data[frame]['car'] = demolished_car_data.copy()
            else:
                start_data = goal_snippet_data[start_frame][entity]
                end_data = goal_snippet_data[end_frame][entity]
                
                for frame in range(start_frame + 1, end_frame):
                    t = (frame - start_frame) / frame_gap
                    if frame not in interpolated_data:
                        interpolated_data[frame] = {}
                    
                    interpolated_data[frame][entity] = interpolate_attributes(start_data, end_data, t)
        
        # Handle end frames
        min_frame = min(goal_snippet_data.keys())
        if entity == 'car':
            # Check remaining frames after interpolation
            remaining_frames = [
                f for f in range(min_frame, max_frame)
                if f not in interpolated_data or 'car' not in interpolated_data[f]
            ]
            if len(remaining_frames) >= 10:  # Car was likely demolished
                # Find the first frame where we have car data (closest to goal)
                first_car_frame = known_frames[0]
                demolished_car_data = goal_snippet_data[first_car_frame]['car'].copy()
                demolished_car_data['sleeping'] = True  # Mark as demolished
                
                # Copy this data to all remaining frames
                for frame in remaining_frames:
                    if frame not in interpolated_data:
                        interpolated_data[frame] = {}
                    interpolated_data[frame]['car'] = demolished_car_data.copy()
            elif len(known_frames) >= 2:  # Normal extrapolation for short gaps
                last_frame = known_frames[-1]
                second_last_frame = known_frames[-2]
                frame_gap = last_frame - second_last_frame
                
                last_data = goal_snippet_data[last_frame]['car']
                second_last_data = goal_snippet_data[second_last_frame]['car']
                
                for frame in remaining_frames:
                    t = (frame - last_frame) / frame_gap
                    if frame not in interpolated_data:
                        interpolated_data[frame] = {}
                    interpolated_data[frame]['car'] = interpolate_attributes(second_last_data, last_data, 1 + t)
        
        else:  # Normal extrapolation for ball
            remaining_frames = [
                f for f in range(min_frame, max_frame)
                if f not in interpolated_data or 'ball' not in interpolated_data[f]
            ]
            
            if remaining_frames and len(known_frames) >= 2:
                last_frame = known_frames[-1]
                second_last_frame = known_frames[-2]
                frame_gap = last_frame - second_last_frame
                
                last_data = goal_snippet_data[last_frame]['ball']
                second_last_data = goal_snippet_data[second_last_frame]['ball']
                
                for frame in remaining_frames:
                    t = (frame - last_frame) / frame_gap
                    if frame not in interpolated_data:
                        interpolated_data[frame] = {}
                    interpolated_data[frame]['ball'] = interpolate_attributes(second_last_data, last_data, 1 + t)
    
    # Handle kickoff positions
    early_frames = [f for f in goal_snippet_data.keys() if f <= 150]
    for entity, default_data in [('ball', KICKOFF_BALL_DATA), ('car', KICKOFF_CAR_DATA)]:
        empty_frames = [
            f for f in early_frames 
            if f not in goal_snippet_data or entity not in goal_snippet_data[f]
        ]
        
        if len(empty_frames) >= 10:
            for frame in empty_frames:
                if frame not in interpolated_data:
                    interpolated_data[frame] = {}
                interpolated_data[frame][entity] = default_data.copy()
                
    return interpolated_data

def interpolate_attributes(start_data: dict, end_data: dict, t: float) -> dict:
    """Helper function to interpolate/extrapolate between two data points.
    
    Args:
        start_data (dict): Starting point data
        end_data (dict): Ending point data
        t (float): Interpolation factor (0-1 for interpolation, >1 for extrapolation)
    """
    if start_data['sleeping']:
        return end_data
    return {
        'sleeping': False,
        'location': {
            'x': start_data['location']['x'] * (1-t) + end_data['location']['x'] * t,
            'y': start_data['location']['y'] * (1-t) + end_data['location']['y'] * t,
            'z': start_data['location']['z'] * (1-t) + end_data['location']['z'] * t
        },
        'rotation': {
            'x': start_data['rotation']['x'] * (1-t) + end_data['rotation']['x'] * t,
            'y': start_data['rotation']['y'] * (1-t) + end_data['rotation']['y'] * t,
            'z': start_data['rotation']['z'] * (1-t) + end_data['rotation']['z'] * t,
            'w': start_data['rotation']['w'] * (1-t) + end_data['rotation']['w'] * t
        },
        'linear_velocity': {
            'x': start_data['linear_velocity']['x'] * (1-t) + end_data['linear_velocity']['x'] * t,
            'y': start_data['linear_velocity']['y'] * (1-t) + end_data['linear_velocity']['y'] * t,
            'z': start_data['linear_velocity']['z'] * (1-t) + end_data['linear_velocity']['z'] * t
        },
        'angular_velocity': {
            'x': start_data['angular_velocity']['x'] * (1-t) + end_data['angular_velocity']['x'] * t,
            'y': start_data['angular_velocity']['y'] * (1-t) + end_data['angular_velocity']['y'] * t,
            'z': start_data['angular_velocity']['z'] * (1-t) + end_data['angular_velocity']['z'] * t
        }
    }

In [24]:
get_relevant_keyframes(goals[2])

[1868]

In [25]:
goals[2]['frame']

2167

In [26]:

def build_dataset_goal(goal_data: dict) -> dict:
    """Builds the dataset for a given goal

    Args:
        goal_data (dict): Data from `goals`

    Returns:
        dict: _description_
    """
    relevant_keyframes = get_relevant_keyframes(goal_data)
    goal_snippet_data = {}
    straddles = len(relevant_keyframes) != 1
    actors = get_actors(relevant_keyframes[-1], goal_data)

    for distance in range(1, FIVE_SECONDS + 1):
        frame = goal_data['frame'] - distance
        if straddles:
            try:
                actors = get_actors(to_recent_keyframe(frame), goal_data)
            except ValueError:
                continue
                # raise ValueError('Failed to get actors')
        actor_data = replay_data['network_frames']['frames'][frame]['updated_actors']
        goal_snippet_data[distance] = get_carball_data(actor_data, actors)
    return goal_snippet_data

In [27]:
import numpy as np

In [28]:
def flatten_game_state(game_state):
    """
    Flatten the game state dictionary into a numpy array.
    For each frame, extracts:
    - Ball: location (3), rotation (4), linear_velocity (3), angular_velocity (3)
    - Car: location (3), rotation (4), linear_velocity (3), angular_velocity (3)
    - Ball sleeping (1)
    - Car sleeping (1)
    Total features per frame: 25
    """
    # Sort frames by frame number and get total count
    sorted_frames = sorted(game_state.items())
    n_frames = len(sorted_frames)
    n_features = 28
    
    # Initialize array with zeros
    flattened = np.zeros(n_frames * n_features, dtype=np.float32)
    
    for frame_idx, (_, frame_data) in enumerate(sorted_frames):
        ball_data = frame_data['ball']
        car_data = frame_data['car']
        
        feature_vector = []
        
        # Ball data
        feature_vector.append(float(ball_data['sleeping']))
        
        # Ball location
        loc = ball_data['location']
        feature_vector.extend([loc['x'], loc['y'], loc['z']])
        
        # Ball rotation
        rot = ball_data['rotation']
        feature_vector.extend([rot['x'], rot['y'], rot['z'], rot['w']])
        
        # Ball linear velocity
        lin_vel = ball_data['linear_velocity']
        feature_vector.extend([lin_vel['x'], lin_vel['y'], lin_vel['z']])
        
        # Ball angular velocity
        ang_vel = ball_data['angular_velocity']
        feature_vector.extend([ang_vel['x'], ang_vel['y'], ang_vel['z']])
        
        # Car data
        feature_vector.append(float(car_data['sleeping']))
        
        # Car location
        loc = car_data['location']
        feature_vector.extend([loc['x'], loc['y'], loc['z']])
        
        # Car rotation
        rot = car_data['rotation']
        feature_vector.extend([rot['x'], rot['y'], rot['z'], rot['w']])
        
        # Car linear velocity
        lin_vel = car_data['linear_velocity']
        if lin_vel is not None:
            feature_vector.extend([lin_vel['x'], lin_vel['y'], lin_vel['z']])
        else:
            feature_vector.extend([0.0, 0.0, 0.0])
        
        # Car angular velocity
        ang_vel = car_data['angular_velocity']
        if ang_vel is not None:
            feature_vector.extend([ang_vel['x'], ang_vel['y'], ang_vel['z']])
        else:
            feature_vector.extend([0.0, 0.0, 0.0])
        
        # Assign the feature vector to the frame
        flattened[frame_idx * n_features:(frame_idx + 1) * n_features] = feature_vector
    
    return flattened

In [29]:
t = build_dataset_goal(goals[1])
# t = interpolate_missing_data(t)
# t = {f: t[f] for f in t.keys() if not f % 5 and f <= 150}
# flatten_game_state(t)
t

{1: {'ball': {'sleeping': False,
   'location': {'x': -130.87, 'y': -5186.66, 'z': 377.84},
   'rotation': {'x': -0.08399462,
    'y': 0.05794846,
    'z': 0.874503,
    'w': 0.47416392},
   'linear_velocity': {'x': 1934.15, 'y': -2071.2, 'z': 238.43},
   'angular_velocity': {'x': -318.3, 'y': 224.35, 'z': 389.15}},
  'car': {'sleeping': False,
   'location': {'x': -247.75, 'y': -5024.59, 'z': 324.22},
   'rotation': {'x': -0.102218285,
    'y': -0.10510454,
    'z': -0.50604194,
    'w': 0.8499565},
   'linear_velocity': {'x': 1349.94, 'y': -1730.92, 'z': 57.91},
   'angular_velocity': {'x': -245.11, 'y': -23.02, 'z': -327.44}}},
 2: {'ball': {'sleeping': False,
   'location': {'x': -211.5, 'y': -5100.32, 'z': 367.45},
   'rotation': {'x': -0.0881648,
    'y': -0.015620676,
    'z': 0.83031607,
    'w': 0.5500529},
   'linear_velocity': {'x': 1936.65, 'y': -2073.85, 'z': 265.86},
   'angular_velocity': {'x': -318.3, 'y': 224.35, 'z': 389.15}}},
 3: {'ball': {'sleeping': False,
   'loc

In [30]:
output = []
for goal in goals:
    raw_data = build_dataset_goal(goal)
    interpolated_data = interpolate_missing_data(raw_data)
    shaved_data = {f: interpolated_data[f] for f in interpolated_data.keys() if not f % 5 and f <= 150}
    flattened_data = flatten_game_state(shaved_data)
    output.append(flattened_data)
output


[array([ 0.00000000e+00, -1.79089996e+02,  4.88075977e+03,  2.62119995e+02,
         6.97472811e-01, -5.01666784e-01, -3.33181202e-01, -3.88397157e-01,
         1.16567004e+03,  2.14079004e+03, -2.59799995e+01,  5.83099976e+02,
         1.15570000e+02,  8.14000015e+01,  0.00000000e+00, -2.27830002e+02,
         4.68808984e+03,  1.95445007e+02,  9.10907507e-01, -5.81642315e-02,
        -1.59640729e-01, -3.59240860e-01,  1.19934497e+03,  1.96222498e+03,
        -2.48750000e+01, -5.06950012e+02, -2.11919998e+02, -2.41550007e+01,
         0.00000000e+00, -3.44559998e+02,  4.57687012e+03,  2.59660004e+02,
         7.89462686e-01, -5.98714113e-01, -2.54014842e-02, -1.32833883e-01,
         1.17077002e+03,  2.15013989e+03,  6.62300034e+01,  5.83099976e+02,
         1.15570000e+02,  8.14000015e+01,  0.00000000e+00, -3.87850006e+02,
         4.41450977e+03,  1.99110001e+02, -5.96906841e-01,  2.53259558e-02,
         3.51350963e-01,  7.20842123e-01,  1.11966003e+03,  2.00285999e+03,
        -2.6

In [ ]:
class JsonToData:
    def __init__(self, fp, seconds_ago = 5):
        self.replay_data = self.load_json(fp)
        self.capture_size = self.replay_data['properties']['RecordFPS'] * seconds_ago
        self.keyframes = [keyframe_data['frame'] for keyframe_data in self.replay_data['keyframes']]
    
    
    def name_to_id(self, name: str) -> int:
        return self.replay_data['names'].index(name)
    
    @staticmethod
    def load_json(fp: str) -> dict:
        with open(fp, 'r') as file:
            return json.load(file)
        
    @staticmethod
    def to_recent_keyframe(frame: int) -> int:
        return min(keyframes, key=lambda keyframe: frame - keyframe if keyframe <= frame else float('inf'))

In [221]:
goals[1]['frame']

2816

In [224]:
replay_data['network_frames']['frames'][2696]

{'time': 103.540184,
 'delta': 0.0333337,
 'new_actors': [],
 'deleted_actors': [],
 'updated_actors': [{'actor_id': 174,
   'stream_id': 42,
   'object_id': 59,
   'attribute': {'RigidBody': {'sleeping': False,
     'location': {'x': -2364.53, 'y': 4611.31, 'z': 16.97},
     'rotation': {'x': 0.0018854814,
      'y': 0.0041890596,
      'z': -0.4515543,
      'w': 0.8922318},
     'linear_velocity': {'x': 784.99, 'y': -1217.24, 'z': 1.02},
     'angular_velocity': {'x': 0.59, 'y': 0.29, 'z': 57.63}}}},
  {'actor_id': 174,
   'stream_id': 50,
   'object_id': 299,
   'attribute': {'Byte': 98}},
  {'actor_id': 148,
   'stream_id': 42,
   'object_id': 59,
   'attribute': {'RigidBody': {'sleeping': False,
     'location': {'x': -3690.2, 'y': 937.96, 'z': 573.44},
     'rotation': {'x': 0.7264567,
      'y': 0.56021667,
      'z': 0.31311247,
      'w': 0.24572042},
     'linear_velocity': {'x': -2130.4, 'y': -2042.91, 'z': -241.51},
     'angular_velocity': {'x': -354.08, 'y': 161.93, 'z':